In [1]:
import keras
import tensorflow as tf
from keras import layers,Model
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout,InputLayer,Flatten,Dense,BatchNormalization,MaxPooling2D,Conv2D,Input,Concatenate,LeakyReLU
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical,normalize
from keras.applications.xception import Xception,preprocess_input
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint,TensorBoard
import numpy as np
import joblib
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm_notebook as tqdm
import warnings
import keras.backend as K
import random
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Loading already preprocessed files.
x_test=joblib.load('/content/drive/MyDrive/T2/x_test.pkl')
x_train=joblib.load('/content/drive/MyDrive/T2/x_train.pkl')
y_test=joblib.load('/content/drive/MyDrive/T2/y_test.pkl')
y_train=joblib.load('/content/drive/MyDrive/T2/y_train.pkl')
y_cv=joblib.load('/content/drive/MyDrive/T2/y_cv.pkl')
x_cv=joblib.load('/content/drive/MyDrive/T2/x_cv.pkl')

In [4]:
print("Shape of Train data {}".format(x_train.shape))
print("Shape of Cv data {}".format(x_cv.shape))
print("Shape of Test data {}".format(x_test.shape))

Shape of Train data (1140, 299, 299, 3)
Shape of Cv data (150, 299, 299, 3)
Shape of Test data (360, 299, 299, 3)


In [5]:
Y_train=to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_cv = to_categorical(y_cv)

### Xception Network

In [6]:
#Greedy Layer-Wise Pretraining Learned from : https://machinelearningmastery.com/greedy-layer-wise-pretraining-tutorial/
# 1. Pre-training the network with train data on initilizing the "imagenet" weights with "include_top=False"
# 2. After training 3 epochs remove the top most layer
# 3. Now add similar top layer and again train and validate


Xception_initial=Xception(include_top=False,
                 weights='imagenet',
                 input_shape=(299,299,3),pooling ='avg',
                 )
#print(Xception_pre_trained.summary())

for layer in Xception_initial.layers:
    layer.trainable = True

x = Xception_initial.output
predicted = Dense(2,activation ='softmax')(x)
model_pretrain = Model(inputs = Xception_initial.input, outputs = predicted)
model_pretrain.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=tf.keras.optimizers.Adam(lr = 0.0002),
          metrics=['accuracy'])
pretraining_Xception =model_pretrain.fit(x_train, Y_train,
                                              verbose=1,
                                              batch_size=32,  
                                              epochs=3)

83697664/83683744 [==============================] - 1s 0us/step
Epoch 1/3
36/36 [==============================] - 101s 2s/step - loss: 0.1493 - accuracy: 0.9421
Epoch 2/3
36/36 [==============================] - 81s 2s/step - loss: 0.0079 - accuracy: 0.9974
Epoch 3/3
36/36 [==============================] - 80s 2s/step - loss: 0.0278 - accuracy: 0.9912


In [7]:
model_pretrain.layers.pop() # Removing topmost layer

In [8]:
%cd /content/drive/MyDrive/T2

/content/drive/MyDrive/T2


In [9]:
tensorboard=TensorBoard(log_dir='./logs', histogram_freq=3)

In [10]:
for layer in model_pretrain.layers:
    layer.trainable = False
x = model_pretrain.output
#x= Dense(64,activation ='relu')(x)
#x = Dropout(0.2)(x)
#x= Dense(64,activation ='relu')(x)
#x = Dropout(0.2)(x)
#x= BatchNormalization()(x) 
predicted = Dense(2,activation ='softmax')(x)
model_finetune1 = Model(inputs = model_pretrain.input, outputs = predicted)
model_finetune1.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=tf.keras.optimizers.Adam(lr = 0.0002),
          metrics=['accuracy'])

finetuning1_XceptionNet=model_finetune1.fit(x_train, Y_train,verbose=1,batch_size=16,
                     epochs=15,validation_data=(x_cv, Y_cv)
                      ,callbacks=[tensorboard]
                     )

Epoch 1/15
72/72 [==============================] - 36s 432ms/step - loss: 0.6515 - accuracy: 0.5184 - val_loss: 0.6774 - val_accuracy: 0.6400
Epoch 2/15
72/72 [==============================] - 22s 303ms/step - loss: 0.6381 - accuracy: 0.9649 - val_loss: 0.6771 - val_accuracy: 0.6067
Epoch 3/15
72/72 [==============================] - 22s 303ms/step - loss: 0.6251 - accuracy: 0.9737 - val_loss: 0.6770 - val_accuracy: 0.5800
Epoch 4/15
72/72 [==============================] - 27s 374ms/step - loss: 0.6125 - accuracy: 0.9807 - val_loss: 0.6767 - val_accuracy: 0.5600
Epoch 5/15
72/72 [==============================] - 22s 305ms/step - loss: 0.6002 - accuracy: 0.9833 - val_loss: 0.6768 - val_accuracy: 0.5467
Epoch 6/15
72/72 [==============================] - 22s 304ms/step - loss: 0.5882 - accuracy: 0.9842 - val_loss: 0.6769 - val_accuracy: 0.5467
Epoch 7/15
72/72 [==============================] - 27s 371ms/step - loss: 0.5765 - accuracy: 0.9851 - val_loss: 0.6772 - val_accuracy: 0.5333

In [11]:
model_finetune1.save("model_finetuned_xception.hdf5")

In [12]:
model_finetune1.evaluate(x_train,Y_train)

36/36 [==============================] - 18s 506ms/step - loss: 0.4879 - accuracy: 0.9904


[0.4879346489906311, 0.9903509020805359]

In [13]:
model_finetune1.evaluate(x_cv,Y_cv)

5/5 [==============================] - 4s 822ms/step - loss: 0.6818 - accuracy: 0.5333


[0.6818447113037109, 0.5333333611488342]

In [14]:
#Unseen data
model_finetune1.evaluate(x_test,Y_test)

12/12 [==============================] - 6s 529ms/step - loss: 0.6394 - accuracy: 0.6806


[0.6393774151802063, 0.6805555820465088]